## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd
from pymystem3 import Mystem

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
df = pd.read_csv('./datasets/data.csv')
df.info()
print('')
print('Количество пустых строк:')
print(df[df['total_income'].isnull() | df['days_employed'].isnull()].count())
print('')
print('Общий трудовой стаж в днях min: {}; max: {}'.format(df['days_employed'].min(), df['days_employed'].max()))
print('')
print('Количество детей:')
print(df['children'].value_counts())
print('')
print('Возраст:')
print(df['dob_years'].value_counts())
print('')
print('Пол:')
print(df['gender'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Количество пустых строк:
children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       

### Вывод

Имеется dataset с 21525 записями и 12 столбцами. Встречаются целочисленные данные, числа с плавающей точкой, а так же строковые данные. Столбец days_employed и total_income имеют пропуски. Столбец days_employed содержит не целочисленные и отрицательные числа, такие результаты являются программной ошибкой, ведь количество дней это неотрицательное целое число. Столбец children имеет невозможное минимальное(-1) и подозрительно часто встречающееся максимальное значение(20), вероятнее всего это человеческая ошибка, т.к. большая часть данных содержит правдивую информацию. В поле dob_years 101 запись с нулевым значением, вероятно часть пользователей не заполняло это поле. А в поле gender имеется артефакт XNA, который тоже указывает на человеческую ошибку, т.к. значение встречается один раз.

Всё вышеперечисленное показывает нам, что поставщик данных предоставляет нам очень грязные данные, с ошибками и пропусками.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
df.info()
df['days_employed'] = df['days_employed'].fillna(0)
print('')
df.info()
print('')
print(df[df['total_income'].isnull()]['income_type'].value_counts())
print('')
income_types = df[df['total_income'].isnull()]['income_type'].unique()

for income_type in income_types:
    try:
        total_income_median = df[df['income_type'] == income_type]['total_income'].median()
        df.loc[df['income_type'] == income_type, 'total_income'] = df.loc[df['income_type'] == income_type, 'total_income'].fillna(total_income_median)
    except:
        print('Что-то пошло не так.')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------   

### Вывод

10% данных имеют пустые значения, это большой процент, поэтому просто удалить эти данные мы не можем, вероятнее всего это програмнная ошибка поставщика данных, либо эти данные сознательно не были указаны пользователями. Столбец days_employed не требуется для ответов на поставленные в проекте вопросы, поэтому мы можем выставить в этом столбце 0 для всех пустых значений. А для total_income мы выберем медианные значения относительно рода деятельности.

### Замена типа данных

In [4]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Для удобство, быстроты и корректности вычеслений мы приводим столбец total_income к целочисленному типу с помощью метода .astype('int)

### Обработка дубликатов

In [5]:
print(df['education'].value_counts())
df['education'] = df['education'].str.lower()
print('')
print(df['education'].value_counts())
df.duplicated().sum()

df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


71

### Вывод

Прежде чем искать дубликаты, надо проверить, что все строковые значения записаны в одном регистре, для этого мы используем .value_counts(), который показывает нам что в столбце education есть одинаковые значения, но записанные в разном регистре. После привидения к единному регистру с помощью метода .duplicated() мы находим 71 дубликат, которые мы удаляем с помощью .drop_dplicates() и затем сбрассываем индекс df.

### Лемматизация

In [6]:
m = Mystem()

def lemmatize_purpose(row):
    return m.lemmatize(row['purpose'])

df['purpose_lemmatized'] = df.apply(lemmatize_purpose, axis = 1)

### Вывод

Для лемматизации столбца purpose была использована библиотека pymystem3. Чтобы провести лемматизацию по всему столбцу была написана функция lemmatize_purpose которая принимает на вход строку и возвращает список лемм, которые вставляются в новый столбец purpose_lemmatized.


### Категоризация данных

In [7]:
from collections import Counter

all_lemmas = df['purpose_lemmatized'].sum()
print(Counter(all_lemmas))

def purpose_category(row):
    if 'недвижимость' in row['purpose_lemmatized'] or 'жилье' in row['purpose_lemmatized']:
        return 'недвижимость'
    if 'автомобиль' in row['purpose_lemmatized']:
        return 'автомобиль'
    if 'образование' in row['purpose_lemmatized']:
        return 'образование'
    if 'операция' in row['purpose_lemmatized']:
        return 'операция'
    if 'свадьба' in row['purpose_lemmatized']:
        return 'свадьба'
    if 'строительство' in row['purpose_lemmatized'] or 'ремонт' in row['purpose_lemmatized']:
        return 'строительство'
    return 'прочее'

df['purpose_category'] = df.apply(purpose_category, axis = 1)
df['purpose_category'].value_counts()

Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_category, dtype: int64

### Вывод

Выделяем список категорий на основе часто встречающихся слов в лемматизированном столбце и устанавливаем категорию с помощью метода purpose_category, который проверяет наличие слова в словаре лемм в столбце purpose_lemmatized.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
df_with_childrens = df[df['children'] > 0]
total_with_childrens = df_with_childrens.shape[0]
total_with_childrens_and_debt = df_with_childrens[df_with_childrens['debt'] == 1].shape[0]
print('Процент людей с детьми, имеющие задолжность: {:.2%}'.format(total_with_childrens_and_debt / total_with_childrens))

print('')
df_without_childrens = df[df['children'] < 1]
total_without_childrens = df_without_childrens.shape[0]
total_without_childrens_and_debt = df_without_childrens[df_without_childrens['debt'] == 1].shape[0]
print('Процент людей без детей, имеющие задолжность: {:.2%}'.format(total_without_childrens_and_debt / total_without_childrens))

Процент людей с детьми, имеющие задолжность: 9.24%

Процент людей без детей, имеющие задолжность: 7.50%


### Вывод

Есть связь между наличием детей и возвратом кредита в срок: процент людей с детьми имеющих задолженность больше, чем людей с задолженностью без детей, но этот процент не настолько занчительный как предпологалось ранее.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
df.groupby('family_status')['debt'].agg(['sum', 'count', 'mean'])

,sum,count,mean
family_status,,,
Не женат / не замужем,274,2813,0.097405
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202


### Вывод

С помощью группировки и метода .agg(['sum', 'count', 'mean']) мы смогли получить количество должников в группе, количество человек в группе, а mean показывает процент должников в группе. Зависимость между семейным положением и возвратом кредита в срок есть, но разница также незначительна, как в случае с наличием или отсутсвием детей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
print(pd.qcut(df['total_income'], 4).value_counts().sort_index(ascending=False))

def income_level(row):
    total_income = row['total_income']
    if total_income > 195549:
        return 'выше среднего'
    if 142594 < total_income <= 195549:
        return 'средний'
    if 107798 < total_income <= 142594:
        return 'ниже среднего'
    return 'низкий'

df['income_level'] = df.apply(income_level, axis = 1)

df.groupby('income_level')['debt'].agg(['sum', 'count', 'mean'])

(195549.0, 2265604.0]    5381
(142594.0, 195549.0]     5239
(107798.0, 142594.0]     5523
(20666.999, 107798.0]    5382
Name: total_income, dtype: int64


,sum,count,mean
income_level,,,
выше среднего,386,5381,0.071734
ниже среднего,483,5523,0.087452
низкий,427,5382,0.079339
средний,445,5239,0.084940


### Вывод

Предварительно категоризовав столбец total_income, с помощью функции income_level(row) и метода apply(), мы получили уровни дохода income_level. Далее с помощью группировки по столбцу income_level и метода .agg(['sum', 'count', 'mean']) мы смогли получить количество должников в группе, количество человек в группе, а mean показывает процент должников в группе. Зависимость между уровнем дохода и возвратом кредита в срок есть, но разница также незначительна.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
df.pivot_table(index='purpose_category', values='debt', aggfunc={'sum', 'mean', 'count'})

,count,mean,sum
purpose_category,,,
автомобиль,4315.0,0.093395,403.0
недвижимость,10840.0,0.072140,782.0
образование,4022.0,0.091994,370.0
свадьба,2348.0,0.079216,186.0


### Вывод

С помощью сводной таблицы мы смогли получить количество должников в группе, количество человек в группе, а mean показывает процент должников в группе. Зависимость между целью кредита и возвратом кредита в срок есть, но разница также незначительна, как и в остальных случаях.

### Шаг 4. Общий вывод

Проанализировав данные, мы наблюдаем, что семейное положение, количество детей, цель и доход влияют на факт погашения кредита, но влияют не значительно, во всех группах был разброс примерно в среднем по 2%.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.